In [1]:
import os, sys
import plotly.express as px
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import darts
from darts.dataprocessing.transformers.boxcox import BoxCox
from darts.models import LightGBMModel, XGBModel, LinearRegressionModel, TFTModel
from darts.metrics import mase, mse, rmse, mae
from darts.dataprocessing.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, RobustScaler   
from darts.dataprocessing.transformers.scaler import Scaler
from darts.utils.missing_values import extract_subseries

from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from torch.optim.lr_scheduler import ReduceLROnPlateau
from pytorch_lightning.callbacks import ModelCheckpoint
import torch
from wandb.xgboost import WandbCallback

import wandb
wandb.login()


import warnings
warnings.filterwarnings('ignore')

# Set seed
np.random.seed(42)

# Set working directory
os.chdir(r"..") # should be the git repo root directory
print("Current working directory: " + os.getcwd())
repo_name = 'net-load-forecasting'
assert os.getcwd()[-len(repo_name):] == "net-load-forecasting", "Working directory is not the git repo root directory"


from utils.utils import *

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: nikolaushouben (wattcast). Use `wandb login --relogin` to force relogin


Current working directory: c:\Users\nik\Desktop\Berkeley_Projects\net-load-forecasting


In [2]:
clean_data_path = os.path.join(os.getcwd(),'data','clean_data')
model_data_path = os.path.join(os.getcwd(),'data','model_data')

In [72]:

def load_data(config):

    '''

    Function to load the data for the different model setups.

    Parameters
    ----------
    config : Config
        Config object with the model setup parameters.

    Returns
    -------
    data : dict
        Dictionary with the data for a specific model setup.


    '''

    df = pd.read_hdf(os.path.join(clean_data_path, "data_net_load_forecasting.h5"), key=f"{config.temp_resolution}min/netload") / 1e3
    df_irr = pd.read_hdf(os.path.join(clean_data_path, "data_net_load_forecasting.h5"), key=f"{config.temp_resolution}min/weather")
    df_irr.rename({'temperature': 'temp_air'}, axis=1, inplace=True)
    df_pv_forecast = pd.read_hdf(os.path.join(model_data_path, "pv_model_results.h5"), key=f"{config.temp_resolution}min/pv_forecast_META-{config.META}") / 1e3

    train_end = int(config.train_ratio * len(df))
    val_begin = int(config.val_ratio* len(df))


    df_train_int = df[:train_end]
    df_val_int = df[-val_begin:]
    df_test_int = df[-val_begin:]

    df_cov_dir_train = df_irr[:train_end]
    df_cov_dir_val = df_irr[-val_begin:]
    df_cov_dir_test = df_irr[-val_begin:]

    df_cov_pv_train = df_pv_forecast[:train_end]
    df_cov_pv_val = df_pv_forecast[-val_begin:]
    df_cov_pv_test = df_pv_forecast[-val_begin:]

    df_train_add = df_train_int + df_cov_pv_train.values
    df_train_add[df_train_add < 0] = 0
    df_val_add = df_val_int + df_cov_pv_val.values
    df_val_add[df_val_add < 0] = 0
    df_test_add = df_test_int + df_cov_pv_test.values
    df_test_add[df_test_add < 0] = 0

    # In this study we are comparing three different model setups: integrated, additive and direct for net load forecasting
    model_setups = {
                    'integrated': {'target': (df_train_int, df_val_int, df_test_int), 'covs': (df_cov_pv_train, df_cov_pv_val, df_cov_pv_test)},
                    'additive': {'target': (df_train_add, df_val_add, df_test_add), 'covs': (None, None, None)},
                    'direct': {'target': (df_train_int, df_val_int, df_test_int), 'covs': (df_cov_dir_train, df_cov_dir_val, df_cov_dir_test)}
                    }


    return model_setups


def darts_data_pipeline(config, model_setups):

    '''

    Function to transform the data into darts.TimeSeries format and apply the data pipeline.

    Parameters
    ----------
    config : Config

    data : dict
        Dictionary with the data for the different model setups.

    Returns
    -------
    piped_data : dict
        Dictionary with the transformed data for the different model setups.

    pipeline : darts.dataprocessing.pipeline.Pipeline
        Pipeline object with the data pipeline.


    '''

    
    data = model_setups[config.model_setup]

    df_train, df_val, df_test = data['target']
    df_cov_train, df_cov_val, df_cov_test = data['covs']
    

    # Into Darts format
    ts_train = darts.TimeSeries.from_dataframe(df_train, freq=str(config.temp_resolution) + 'min')
    ts_train = extract_subseries(ts_train)
    ts_val = darts.TimeSeries.from_dataframe(df_val, freq=str(config.temp_resolution) + 'min')
    ts_val = extract_subseries(ts_val)
    ts_test = darts.TimeSeries.from_dataframe(df_test, freq=str(config.temp_resolution) + 'min')
    ts_test = extract_subseries(ts_test)

    if config.model_setup == 'additive':
        ts_cov_train = None
        ts_cov_val = None
        ts_cov_test = None
    else:
        ts_cov_train = darts.TimeSeries.from_dataframe(df_cov_train, freq=str(config.temp_resolution) + 'min')
        ts_cov_val = darts.TimeSeries.from_dataframe(df_cov_val, freq=str(config.temp_resolution) + 'min')
        ts_cov_test = darts.TimeSeries.from_dataframe(df_cov_test, freq=str(config.temp_resolution) + 'min')

    # Reviewing subseries to make sure they are long enough

    min_len = config.n_lags + config.n_ahead

    ts_train_, ts_cov_train = review_subseries(ts_train, min_len, ts_cov_train)
    ts_val_, ts_cov_val = review_subseries(ts_val,  min_len, ts_cov_val)
    ts_test_, ts_cov_test = review_subseries(ts_test, min_len, ts_cov_test)



    # Load pipeline
    pipeline = Pipeline( # missing values have been filled in the 'data_prep.ipynb'
                        [
                        Scaler(MinMaxScaler()),
                        ]
                        )
    
    ts_train_piped = pipeline.fit_transform(ts_train_)
    ts_val_piped = pipeline.transform(ts_val_)
    ts_test_piped = pipeline.transform(ts_test_)

    if config.model_setup == 'additive':
        ts_cov_train_piped = None
        ts_cov_val_piped = None
        ts_cov_test_piped = None

    else:  
        # Future Covariate Pipeline
        pipeline_weather = Pipeline([Scaler(MinMaxScaler())])
        ts_cov_train_piped = pipeline_weather.fit_transform(ts_cov_train)
        ts_cov_val_piped = pipeline_weather.transform(ts_cov_val)
        ts_cov_test_piped = pipeline_weather.transform(ts_cov_test)

    # getting the index of the longest subseries, to be used for evaluation later
    longest_ts_val_idx = get_longest_subseries_idx(ts_val_piped)
    longest_ts_test_idx = get_longest_subseries_idx(ts_test_piped)


    piped_data = {'target': (ts_train_piped, ts_val_piped[longest_ts_val_idx], ts_test_piped[longest_ts_test_idx]),
                'covs': (ts_cov_train_piped, ts_cov_val_piped, ts_cov_test_piped),
                'target_inversed': (ts_train, ts_val[longest_ts_val_idx], ts_test[longest_ts_test_idx])}

    return piped_data, pipeline




### Iterating through models

In [51]:
def build_config(config_dataset):

    '''
    
    Takes a config_dataset dictionary and builds a config object from it, deriving the rest of the parameters from the config_dataset.

    '''

    config = Config().from_dict(config_dataset)
    config.temp_resolution = 15 # in minutes
    config.horizon_in_hours = 24 + 36 if config.METER == '2' else 36 # in hours, 24 for the data gap in METER-2 and 36 for the day-ahead forecast horizon
    config.timestep_encoding = ["hour", "minute"] if config.temp_resolution == 1 else ['quarter']
    config.datetime_encoding =  {
                        "cyclic": {"future": config.timestep_encoding, 'past': config.timestep_encoding}, 
                        "position": {"future": ["relative",], 'past': ["relative",]},
                        "datetime_attribute": {"future": ["dayofweek", "week"], 'past': ["dayofweek", "week"]},
                } if config.use_datetime_encoding else None

    config.timesteps_per_hour = int(60 / config.temp_resolution)
    config.n_lags = config.lookback_in_hours * config.timesteps_per_hour
    config.n_ahead = config.horizon_in_hours * config.timesteps_per_hour
    config.eval_stride = int(np.sqrt(config.n_ahead)) # evaluation stride, how often to evaluate the model, in this case we evaluate every n_ahead steps
    
    return config

In [52]:
for METER in ['1','2']:


config_run = {
'model_setups_names': [
                            'additive',
                            'integrated',
                            'direct'
                        ],
'model_setup': None,
'METER': '1',
'META': '1',
'train_ratio': 0.3,
'val_ratio': 0.1,
'lookback_in_hours' : 24,
'liklihood': None,
'holiday': True,
'use_datetime_encoding': True,
'boxcox': False,
'eval_metrics' : [mae, mse, rmse],
'evaluation_set': 'val',
'wandb': True,
}

df_metrics = train_all(config_run)



#TODO 
# 1) Reference peristence benchmark
# 2) Make train and test set cut offs explicit with datetime
# 3) Add more metrics: imbalance costs


In [74]:

scores_dict_meter = {}


for METER in ['1','2']:

    scores_dict_train_ratio = {}
    for train_ratio in [0.3, 0.8]:

        config_run = {
        'model_setups_names': [
                                    'additive',
                                    'integrated',
                                    'direct'
                                ],
        'model_setup': None,
        'METER': METER,
        'META': '1',
        'train_ratio': train_ratio,
        'val_ratio': 0.1,
        'lookback_in_hours' : 24,
        'liklihood': None,
        'holiday': True,
        'use_datetime_encoding': False,
        'boxcox': False,
        'eval_metrics' : [mae, mse, rmse],
        'evaluation_set': 'val',
        'wandb': True,
        }

        df_metrics = train_all(config_run)

        scores_dict_train_ratio[train_ratio] = df_metrics

    scores_dict_meter[METER] = scores_dict_train_ratio
    




#TODO 
# 1) Reference peristence benchmark
# 2) Make train and test set cut offs explicit with datetime
# 3) Add more metrics: imbalance costs


Running model setup: additive
Training model
Evaluating on validation set
Running model setup: integrated
Training model
Evaluating on validation set
Running model setup: direct
Training model
Evaluating on validation set


Running model setup: additive
Training model
Evaluating on validation set
Running model setup: integrated
Training model
Evaluating on validation set
Running model setup: direct
Training model
Evaluating on validation set


Running model setup: additive
Training model
Evaluating on validation set
Running model setup: integrated
Training model
Evaluating on validation set
Running model setup: direct
Training model
Evaluating on validation set


Running model setup: additive
Training model
Evaluating on validation set
Running model setup: integrated
Training model
Evaluating on validation set
Running model setup: direct
Training model
Evaluating on validation set


In [73]:
def train_all(config_run):

    config = build_config(config_run)
    
    if config.wandb:
        wandb.init(project="net-load-forecasting", name=f"META-{config.META}_METER-{config.METER}-train_ratio-{config.train_ratio}")
        wandb.config.update(config.data)

    predictions_per_model = {}
    model_setups_names = config.model_setups_names
    eval_set_idx = {'train': 0, 'val': 1, 'test': 2}
    for model_setup in model_setups_names:

        print(f'Running model setup: {model_setup}')
        config.model_setup = model_setup


        model_setups_data = load_data(config)
        piped_data, pipeline = darts_data_pipeline(config, model_setups_data)
        ts_train_piped, ts_val_piped, ts_test_piped = piped_data['target']
        ts_cov_train_piped, ts_cov_val_piped, ts_cov_test_piped = piped_data['covs']
        trg_train_inversed, trg_val_inversed, trg_test_inversed = piped_data['target_inversed']


        print('Training model')
        model = LinearRegressionModel(lags=config.n_lags,
                        lags_future_covariates= None if config.model_setup == 'additive' else [0],
                        add_encoders=config.datetime_encoding   , 
                        output_chunk_length=config.n_ahead, 
                        likelihood=None,
                        random_state=42
                        )

        model.fit(ts_train_piped, future_covariates = ts_cov_train_piped)

        print('Evaluating on validation set')
        predictions, _ = predict_testset(config, model, 
                                        ts_val_piped, 
                                        ts_cov_val_piped,
                                        pipeline,
                                        )
        
            # subtracting the covariates from the predictions, since we are predicting the net load
        if config.model_setup == 'additive':
            predictions -= model_setups_data['integrated']['covs'][eval_set_idx[config.evaluation_set]].reindex(predictions.index).values

        predictions.columns = ['prediction_'+model_setup]
        predictions_per_model[model_setup] = predictions
        
    if config.evaluation_set == 'val':
        persistance = trg_val_inversed.pd_dataframe().shift(config.timesteps_per_hour * 24)
    else:
        persistance = trg_test_inversed.pd_dataframe().shift(config.timesteps_per_hour * 24)

    persistance.columns = ['24h_persistance']
    predictions_per_model['24h_persistance'] = persistance


    df_metrics = format_and_log_predictions(config, predictions_per_model, trg_val_inversed, trg_test_inversed)

    if config.wandb:
        wandb.finish()

    return df_metrics


def format_and_log_predictions(config, predictions_per_model, trg_val_inversed, trg_test_inversed):

    '''
    Evaluates the predictions on the validation set using the evaluation metrics specified in the config.

    Parameters
    ----------
    df_compare : pandas.DataFrame
        DataFrame with the predictions and the ground truth values.
    config : Config
        Config object with the model setup parameters.

    Returns
    -------
    df_metrics : pandas.DataFrame
        DataFrame with the evaluation metrics for each model.


    '''

    if config.evaluation_set == 'val':
        df_predictions = pd.concat(predictions_per_model.values(), axis=1)
        df_compare = pd.merge(trg_val_inversed.pd_dataframe(), df_predictions, left_index=True, right_index=True)
    else:
        df_predictions = pd.concat(predictions_per_model.values(), axis=1)
        df_compare = pd.merge(trg_test_inversed.pd_dataframe(), df_predictions, left_index=True, right_index=True)


    fig = px.line(df_compare, x=df_compare.index, y=df_compare.columns)

    fig.update_layout(
        title=f"Predictions for {config.evaluation_set} set",
        xaxis_title="Time",
        yaxis_title="Net Load [kW]",
        legend_title="Legend Title"

    )

    if config.wandb: 
        wandb.log({"predictions": fig})
    else:
        fig.show()
    


    ts_gt = darts.TimeSeries.from_dataframe(df_compare.iloc[:,[0]])
    df_metrics = pd.DataFrame(index = df_compare.columns[1:], columns = [metric.__name__ for metric in config.eval_metrics])
    for col in df_compare.columns[1:]:
        ts_pred = darts.TimeSeries.from_dataframe(df_compare[[col]])
        for metric in config.eval_metrics:
            df_metrics.loc[col, metric.__name__] = metric(ts_gt, ts_pred)

    if config.wandb:
        wandb.log({"metrics": wandb.Table(dataframe=df_metrics.reset_index())})

    return df_metrics



### Wandb Transformer Tuning

In [76]:
from darts.models import TFTModel
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from torch.optim.lr_scheduler import ReduceLROnPlateau





def train_tft():


    config_ = build_config(config_run)

    wandb.init()
    config = wandb.config
    config.update(config_.data)

    print('config updated')
    model_setups_data = load_data(config)
    piped_data, pipeline = darts_data_pipeline(config, model_setups_data)
    ts_train_piped, ts_val_piped, ts_test_piped = piped_data['target']
    ts_cov_train_piped, ts_cov_val_piped, ts_cov_test_piped = piped_data['covs']
    trg_train_inversed, trg_val_inversed, trg_test_inversed = piped_data['target_inversed']


    optimizer_kwargs = {}
    try:
        optimizer_kwargs['lr'] = config.lr
    except:
        optimizer_kwargs['lr'] = 1e-3

    pl_trainer_kwargs = {
    'max_epochs': 20,
    'accelerator': 'gpu',
    'devices': [0],
    'callbacks': [EarlyStopping(monitor='val_loss', patience=5, mode='min')],
    'logger': WandbLogger(log_model='best'),
    }

    lr_scheduler_kwargs = {
        'patience': 2,
        'factor': 0.5,
        'min_lr': 1e-5,
        'verbose': True
        }


    model = TFTModel(
    input_chunk_length=config.n_lags,
    output_chunk_length=config.n_ahead,
    hidden_size=config.hidden_size,
    batch_size=config.batch_size,
    lr_scheduler_cls = ReduceLROnPlateau,
    lr_scheduler_kwargs = lr_scheduler_kwargs,
    optimizer_kwargs = optimizer_kwargs,
    pl_trainer_kwargs = pl_trainer_kwargs)
    
    model.fit(ts_train_piped, future_covariates = ts_cov_train_piped, val_series=ts_val_piped, val_future_covariates =ts_cov_val_piped)

    print('Evaluating on validation set')
    predictions, scores = predict_testset(config, model, 
                                    ts_val_piped, 
                                    ts_cov_val_piped,
                                    pipeline,
                                    )

    predictions.columns = ['prediction_'+model_setup]

    df_compare = pd.merge(trg_val_inversed.pd_dataframe(), df_predictions, left_index=True, right_index=True, how = 'left')
    fig = px.line(df_compare)


    wandb.log({'fig': fig})
    wandb.log({'val_rmse': scores['rmse']})
    wandb.finish()

In [77]:
train_tft()

wandb: WARNING Config item 'model_setup' was locked by 'sweep' (ignored update).


config updated


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name                              | Type                             | Params
----------------------------------------------------------------------------------------
0  | train_metrics                     | MetricCollection                 | 0     
1  | val_metrics                       | MetricCollection                 | 0     
2  | input_embeddings                  | _MultiEmbedding                  | 0     
3  | static_covariates_vsn             | _VariableSelectionNetwork        | 0     
4  | encoder_vsn                       | _VariableSelectionNetwork        | 25.0 K
5  | decoder_vsn                       | _VariableSelectionNetwork        | 11.9 K
6  | static_context_grn                | _GatedResidualNetwork            | 1.1 M 
7  | static_context_hidden_encoder_grn | _GatedResid

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
wandb: WARNING Config item 'hidden_size' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'num_attention_heads' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'full_attention' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'dropout' was locked by 'sweep' (ignored update).


Evaluating on validation set


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
wandb: WARNING Config item 'hidden_size' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'num_attention_heads' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'full_attention' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'dropout' was locked by 'sweep' (ignored update).
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
wandb: WARNING Config item 'hidden_size' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'num_attention_heads' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'full_attention' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 

AttributeError: <class 'wandb.sdk.wandb_config.Config'> object has no attribute 'eval_metrics'

### Testinfg

In [28]:


model_setups_names = ['integrated',
                        'additive',
                        'direct'
                      ]

config_run = {
'model_setup': 'integrated',
'METER': '1',
'META': '1',
'train_ratio': 0.2,
'val_ratio' : 0.1,
'lookback_in_hours' : 24,
'liklihood': None,
'holiday': True,
'use_datetime_encoding': False,
'boxcox': False,
#'eval_metrics' : [mae, mse, rmse],
'evaluation_set': 'val'
}

eval_set_idx = {'train': 0, 'val': 1, 'test': 2}




sweep_config = {
    'method': 'bayes', #grid, random
    'metric': {
        'name': 'val_rmse',
        'goal': 'minimize'
    },
    'parameters': {
        'model_setup': {
            'values': ['integrated']
        },
        'batch_size': {
            'values': [32, 64, 128]
        },
        'lr': {
            'values': [3e-4, 1e-3, 1e-4, 1e-5]
        },
        'full_attention': {
            'values': [True, False]
        },
        'dropout': {
            'values': [0.1, 0.2, 0.3]
        },
        'num_attention_heads': {
            'values': [3,4,5,6]
        },
        'hidden_size': {
            'values': [512, 1024, 2048]
        }
    }
}



sweep_id = wandb.sweep(sweep_config, project="net-load-forecasting")

wandb.agent(sweep_id, function=train_tft, count=1)

        



Create sweep with ID: sy2xu7fo
Sweep URL: https://wandb.ai/wattcast/net-load-forecasting/sweeps/sy2xu7fo


wandb: Agent Starting Run: r4v1v0k5 with config:
wandb: 	batch_size: 128
wandb: 	dropout: 0.1
wandb: 	full_attention: False
wandb: 	hidden_size: 512
wandb: 	lr: 0.0001
wandb: 	model_setup: integrated
wandb: 	num_attention_heads: 5


wandb: WARNING Config item 'model_setup' was locked by 'sweep' (ignored update).


config updated


wandb: ERROR Run r4v1v0k5 errored: IndexError('list index out of range')
